In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import io

In [37]:
base_url = r"https://www.sec.gov/"
fullIndex_url = r"https://www.sec.gov/Archives/edgar/full-index/"


normal_url = r"https://www.sec.gov/Archives/edgar/data/1265107/0001265107-19-000004.txt"
json_url = normal_url.replace('-','').replace('.txt','/index.json')

documents_url = r"https://www.sec.gov/Archives/edgar/data/1265107/000126510719000004/index.json"

content = requests.get(documents_url).json()

for file in content['directory']['item']:
    
    # Grab the filing summary and create a new url leading to the file so we can download it.
    if file['name'] == 'FilingSummary.xml':

        xml_summary = base_url + content['directory']['name'] + "/" + file['name']
        
        print('-' * 100)
        print('File Name: ' + file['name'])
        print('File Path: ' + xml_summary)
        
        
# define a new base url that represents the filing folder. This will come in handy when we need to download the reports.
base_url = xml_summary.replace('FilingSummary.xml', '')

# request and parse the content
content = requests.get(xml_summary).content
soup = BeautifulSoup(content, 'lxml')

# find the 'myreports' tag because this contains all the individual reports submitted.
reports = soup.find('myreports')

# I want a list to store all the individual components of the report, so create the master list.
master_reports = []

# loop through each report in the 'myreports' tag but avoid the last one as this will cause an error.
for report in reports.find_all('report')[:-1]:

    # let's create a dictionary to store all the different parts we need.
    report_dict = {}
    report_dict['name_short'] = report.shortname.text
    report_dict['name_long'] = report.longname.text
    report_dict['position'] = report.position.text
    report_dict['category'] = report.menucategory.text
    report_dict['url'] = base_url + report.htmlfilename.text
    report_dict['xml'] = base_url + report['instance']

    # append the dictionary to the master list.
    master_reports.append(report_dict)
    
# create the list to hold the statement urls
statements_url = []
xml_url = []

# define the statements we want to look for.
item1 = r"Consolidated Balance Sheets"
item2 = r"Consolidated Statements of Operations and Comprehensive Income (Loss)"
item3 = r"Consolidated Statements of Cash Flows"
item4 = r"Consolidated Statements of Stockholder's (Deficit) Equity"

# store them in a list.
report_list = [item1, item2, item3, item4]

for report_dict in master_reports:
    
    # if the short name can be found in the report list.
    if report_dict['name_short'] in report_list:
        
#         # print some info and store it in the statements url.
#         print('-'*100)
#         print(report_dict['name_short'])
#         print(report_dict['url'])
#         print(report_dict['xml'])
        
        statements_url.append(report_dict['url'])
        xml_url.append(report_dict['xml'])
        
for statement in [statements_url[0]]: # Should do all statements, but focus on balance sheet for now

    # request the statement file content
    content = requests.get(statement).content
    report_soup = BeautifulSoup(content, 'html')

    data = {}

    for row in report_soup.table.find_all('tr'):
        
        if row.th:
            cols = row.find_all('th')
            data['name'] = cols[0].text.strip()
            data['years'] = [element.text.strip() for element in cols[1:]]
            continue
            

        if row.find_all('strong'): # Means a section head
            cols = row.find_all('td')
            key = cols[0].text.strip()
            data[key] = {}

        if row.find_all('strong').__len__() == 0:
            cols = [element.text.strip() for element in row.find_all('td')]
            subcategory = cols[0]
            data[key][subcategory] = cols[1:]
            
with open('./data/bs/balance.txt', 'w') as file:
    file.write(json.dumps(data))

In [307]:
quarter['name']

'QTR4'

In [318]:
raw_df

,Company Name,Form Type,Date Filed,Filename
CIK,,,,
860585,RBS PARTNERS L P /CT,13FCONP,1993-02-11,edgar/data/860585/9999999997-04-035713.txt
880794,MERRILL LYNCH LIFE VARIABLE ANNUITY SEPARATE A...,NSAR-B,1993-02-26,edgar/data/880794/9999999997-05-050433.txt
926688,SMITH THOMAS W,13F-HR,1993-02-12,edgar/data/926688/9999999997-05-015654.txt
94673,STORAGE TECHNOLOGY CORP,CERTNYS,1993-02-24,edgar/data/94673/9999999997-05-037760.txt


In [317]:
raw_df[raw_df['Form Type'].str.match(r"10-[QK]$")]["Form Type"].str.extract(r"([QK])")

,0


In [321]:
total_df

,2003QTR3,2003QTR2,2003QTR1,2002QTR4,2002QTR3,2002QTR2,2002QTR1,2001QTR4,2001QTR3,2001QTR2,...,1995QTR4,1995QTR3,1995QTR2,1995QTR1,1994QTR4,1994QTR3,1994QTR2,1994QTR1,1993QTR4,1993QTR3
CIK,,,,,,,,,,,,,,,,,,,,,
20,Q,Q,K,Q,Q,Q,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1750,K,Q,Q,Q,K,Q,Q,Q,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,Q,Q,K,Q,Q,Q,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1923,"K, Q",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2034,K,Q,Q,Q,K,Q,Q,Q,K,Q,...,Q,K,Q,Q,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237941,Q,Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1241199,Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1243800,Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [326]:
reference_df = pd.DataFrame()
total_raw = []

fullIndex_url = r"https://www.sec.gov/Archives/edgar/full-index/"

fullIndex = json.loads(requests.get(fullIndex_url + r"index.json").content)

for year in fullIndex['directory']['item']:
    
    year_url = fullIndex_url + year['href']
    yearIndex = json.loads(requests.get(year_url + r"index.json").content)
    
    for quarter in yearIndex['directory']['item']:
        
        print(year['name']+quarter['name'])
        
        quarter_url = year_url + quarter['href']
        
        
        raw_df = pd.read_csv(io.StringIO(requests.get(quarter_url + r"master.idx").content.decode('utf-8')),
                    delimiter = '|',
                    skiprows = [0,1,2,3,4,5,6,7,8,10],
                    index_col=0
                   )
        
        total_raw.append(raw_df)
        
        try:
            df = raw_df[raw_df['Form Type'].str.match(r"10-[QK]$")]["Form Type"].str.extract(r"([QK])")
            df = df.groupby("CIK").aggregate({0: ', '.join})
            df.columns = [ year['name']+ quarter['name']]
            reference_df = df.join(reference_df)

        except:
            print(year['name']+quarter['name'] + " did not contain data.")
        
        

        

        
        
    
    
# year = json.loads(requests.get(fullIndex_url + fullIndex['directory']['item'][0]['href'] + r"index.json").content)
# quarter = json.loads(requests.get(fullIndex + year['directory']['item'][0]['href'] +  + r"index.json").content)

1993QTR1
1993QTR1 did not contain data.
1993QTR2
1993QTR2 did not contain data.
1993QTR3
1993QTR4
1994QTR1
1994QTR2
1994QTR3
1994QTR4
1995QTR1
1995QTR2
1995QTR3
1995QTR4
1996QTR1
1996QTR2
1996QTR3
1996QTR4
1997QTR1
1997QTR2
1997QTR3
1997QTR4
1998QTR1
1998QTR2
1998QTR3
1998QTR4
1999QTR1
1999QTR2
1999QTR3
1999QTR4
2000QTR1
2000QTR2
2000QTR3
2000QTR4
2001QTR1
2001QTR2
2001QTR3
2001QTR4
2002QTR1
2002QTR2
2002QTR3
2002QTR4
2003QTR1
2003QTR2
2003QTR3
2003QTR4
2004QTR1
2004QTR2
2004QTR3
2004QTR4
2005QTR1
2005QTR2
2005QTR3
2005QTR4
2006QTR1
2006QTR2
2006QTR3
2006QTR4
2007QTR1
2007QTR2
2007QTR3
2007QTR4
2008QTR1
2008QTR2
2008QTR3
2008QTR4
2009QTR1
2009QTR2
2009QTR3
2009QTR4
2010QTR1
2010QTR2
2010QTR3
2010QTR4
2011QTR1
2011QTR2
2011QTR3
2011QTR4


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 13013584: invalid continuation byte

In [310]:
total_df

,1994QTR4,1994QTR3,1994QTR2,1994QTR1
CIK,,,,
1750,Q,K,Q,Q
1800,Q,Q,Q,K
2024,Q,Q,Q,K
2145,Q,NaN,NaN,NaN
2488,Q,Q,Q,K
...,...,...,...,...
925503,Q,Q,NaN,NaN
925504,Q,Q,NaN,NaN
925692,Q,NaN,NaN,NaN


In [296]:
total_df = pd.DataFrame()
for index,df in enumerate(summary_df):
    print(index)

    df = df.set_index(["CIK"])
    df = df[df['Form Type'].str.match(r"10-[QK]$")]["Form Type"].str.extract(r"([QK])")
    df = df.groupby("CIK").aggregate({0: ', '.join})
    df.columns = ['1994QTR' + str(index)]
#     df.columns=[]
    total_df=df.join(total_df)

0
1
2
3


,1994QTR3,1994QTR2,1994QTR1,1994QTR0
CIK,,,,
1750,Q,K,Q,Q
1800,Q,Q,Q,K
2024,Q,Q,Q,K
2145,Q,NaN,NaN,NaN
2488,Q,Q,Q,K
...,...,...,...,...
925503,Q,Q,NaN,NaN
925504,Q,Q,NaN,NaN
925692,Q,NaN,NaN,NaN


In [284]:
df = summary_df[0]
df = df.set_index(["CIK"])
df = df[df['Form Type'].str.match(r"10-[QK]$")]["Form Type"].str.extract(r"([QK])")
df = df.groupby("CIK").aggregate({0: ', '.join})
df.columns=['1994']

df1 = summary_df[1]
df1 = df1.set_index(["CIK"])
df1 = df1[df1['Form Type'].str.match(r"10-[QK]$")]["Form Type"].str.extract(r"([QK])")
df1 = df1.groupby("CIK").aggregate({0: ', '.join})


In [290]:
df1.join(df)

,0,1994
CIK,,
1750,Q,Q
1800,Q,K
2024,Q,K
2488,Q,K
2648,Q,K
...,...,...
914179,Q,Q
914183,Q,Q
916849,Q,NaN


In [268]:
df1[0]

CIK
1750      Q
1800      Q
2024      Q
2488      Q
2648      Q
         ..
914179    Q
914183    Q
916849    Q
919096    Q
922404    Q
Name: 0, Length: 2078, dtype: object

In [259]:
test= filed_10[filed_10["Form Type"]=="10-K"].set_index('CIK').join(
    filed_10[filed_10["Form Type"]=="10-Q"].set_index('CIK'),
    on='CIK',
    lsuffix='K',
    rsuffix='Q'
)

In [98]:
pd.concat([filed_10["Form Type"]=="10-K", filed_10["Form Type"]=="10-Q"], axis=1,).set_index(filed_10.CIK)

,Form Type,Form Type
CIK,,
100240,True,False
100493,False,True
100726,False,True
100783,True,False
100826,True,False
...,...,...
99250,True,False
99359,False,True
99780,False,True


In [ ]:
        # Read the master idx, (log of all entries sorted by CIK) by url
        df = pd.read_csv(
            io.StringIO(requests.get(quarter_url + r"master.idx").content.decode('utf-8')),
            delimiter= '|', 
            skiprows= [0,1,2,3,4,5,6,7,8,10], # skipping leading rows, N.B. row 9 contains headers
        )
        

        # Pull any entries containing 10-K or 10-Q and look at columns CIK and Form Type
        filed_10 = df.loc[df['Form Type'].str.match(r"10-[QK]$"),["CIK","Form Type"]] 

        # Create a new database out of these entries, and combine it with the complete registry (e.g. named 1993QTR1 ...)
        summary_df.append(pd.DataFrame(
            data = filed_10["Form Type"].values, 
            index= filed_10["CIK"], 
            columns = [(year['name'] + quarter['name'])]
        ))